In [1]:
#####THIS CODE aims to run TPOT on regressor data#####

#Adjustable parameters
max_time_mins = 240 #Time (mins) use per running each TPOT pipeline. Set = None for no limitations
main_path = '/home/korawich/Desktop/AutoML/automl_ddmut/'

In [2]:
#Datafile #change here to use different data
data_path = main_path + 'AutoML/Dataset/DDMut_data/'
gbsig_path = main_path + 'Dataset/DDMut_data/Graph-based/'
result_path = main_path + 'Dataset/DDMut_data/Results/'
final_path = main_path + 'Dataset/DDMut_data/Final/'

#train_info_path  = final_path + 'train_info.csv'
#test_1_info_path = final_path + 'test_1_info.csv'
#test_2_info_path = final_path + 'test_2_info.csv'
#test_3_info_path = final_path + 'test_3_info.csv'

train_X_path  = final_path + 'final_train_X.csv'
test_1_X_path = final_path + 'final_test_1_X.csv'
test_2_X_path = final_path + 'final_test_2_X.csv'
test_3_X_path = final_path + 'final_test_3_X.csv'

train_y_path  = final_path + 'train_y.csv'
test_1_y_path = final_path + 'test_1_y.csv'
test_2_y_path = final_path + 'test_2_y.csv'
test_3_y_path = final_path + 'test_3_y.csv'

In [3]:
import numpy as np
import pandas as pd
from tpot import TPOTRegressor
from scipy.stats import pearsonr

import warnings
warnings.filterwarnings('ignore')

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import mean_squared_error, r2_score
from tpot.export_utils import set_param_recursive

In [5]:
#Code path 
alg_path = main_path + 'Algorithm/TPOT/'

In [6]:
X_train = pd.read_csv(train_X_path, index_col=0)
X_test_1 = pd.read_csv(test_1_X_path, index_col=0)
X_test_2 = pd.read_csv(test_2_X_path, index_col=0)
X_test_3 = pd.read_csv(test_3_X_path, index_col=0)

y_train = pd.read_csv(train_y_path, index_col=0)
y_test_1 = pd.read_csv(test_1_y_path, index_col=0)
y_test_2 = pd.read_csv(test_2_y_path, index_col=0)
y_test_3 = pd.read_csv(test_3_y_path, index_col=0)

In [7]:
#Check size of datasets
print('X_train.shape = ', X_train.shape)
print('X_test_1.shape = ', X_test_1.shape)
print('X_test_2.shape = ', X_test_2.shape)
print('X_test_3.shape = ', X_test_3.shape)

X_train.shape =  (9028, 896)
X_test_1.shape =  (552, 896)
X_test_2.shape =  (1304, 896)
X_test_3.shape =  (2024, 896)


In [8]:
#load list of features
feat_list_path = main_path + 'Dataset/DDMut_data/Feat_list/'
gbsig_feat_list_path = feat_list_path + 'gbsig_feat_list.csv'
seq_feat_list_path = feat_list_path + 'seq_feat_list.csv'
str_feat_list_path = feat_list_path + 'str_feat_list.csv'

all_feat_result_path = result_path + 'tpot_default_params.csv'
gbsig_feat_result_path = result_path + 'tpot_default_params_gbsig.csv'
seq_feat_result_path = result_path + 'tpot_default_params_seq.csv'
str_feat_result_path = result_path + 'tpot_default_params_str.csv'

gbs_feat = pd.read_csv(gbsig_feat_list_path, index_col=0)
seq_feat = pd.read_csv(seq_feat_list_path, index_col=0)
str_feat = pd.read_csv(str_feat_list_path, index_col=0)
all_feat = pd.concat([seq_feat, str_feat, gbs_feat], ignore_index=True)

In [9]:
features_sets = [
    ('all_feat', all_feat, all_feat_result_path),
    ('seq_feat', seq_feat, seq_feat_result_path),
    ('str_feat', str_feat, str_feat_result_path),
    ('gbs_feat', gbs_feat, gbsig_feat_result_path)
]

2. TPOT

In [10]:
#The default parameter is set to be the same as in TPOT_paper
results = []
for feat_name, feat_list, result_path in features_sets:
    print('running - feat_name: ', feat_name)
    X_train_sel  = X_train[feat_list.values.flatten()]
    X_test_1_sel = X_test_1[feat_list.values.flatten()]
    X_test_2_sel = X_test_2[feat_list.values.flatten()]
    X_test_3_sel = X_test_3[feat_list.values.flatten()]
    
    tpot = TPOTRegressor(generations=100, population_size=100,
                             offspring_size=None, mutation_rate=0.9,
                             crossover_rate=0.1,
                             scoring='neg_mean_squared_error', cv=5,
                             subsample=1.0, n_jobs=-1,
                             max_time_mins=max_time_mins, max_eval_time_mins=5, #Normally, max_time_mins=None
                             random_state=42, config_dict=None, #The config dict is none, use the default
                             template=None,
                             warm_start=False,
                             memory=None,
                             use_dask=False, #Parallel training
                             periodic_checkpoint_folder=None,
                             early_stop=None,
                             verbosity=2,
                             disable_update_check=False)

    #export pipeline (TPOPT)
    tpot.fit(X_train_sel, y_train)
    pipeline_path = alg_path + 'tpot_exported_pipeline.py'
    tpot.export(pipeline_path)

    exported_pipeline = tpot.fitted_pipeline_
    set_param_recursive(exported_pipeline.steps, 'random_state', 42)
    exported_pipeline.fit(X_train_sel, y_train)

    #Test in test dataset
    y_pred_test_1 = exported_pipeline.predict(X_test_1_sel)
    rmse_1 = np.sqrt(mean_squared_error(y_test_1, y_pred_test_1))
    r2_1 = r2_score(y_test_1, y_pred_test_1)
    pearson_corr_1, _ = pearsonr(y_test_1.values.flatten(), y_pred_test_1.flatten())
    print('score for test 1 - RMSE = ', rmse_1, ' - R2 = ', r2_1, ' - Pearson = ', pearson_corr_1)
    
    y_pred_test_2 = exported_pipeline.predict(X_test_2_sel)
    rmse_2 = np.sqrt(mean_squared_error(y_test_2, y_pred_test_2))
    r2_2 = r2_score(y_test_2, y_pred_test_2)
    pearson_corr_2, _ = pearsonr(y_test_2.values.flatten(), y_pred_test_2.flatten())
    print('score for test 2 - RMSE = ', rmse_2, ' - R2 = ', r2_2, ' - Pearson = ', pearson_corr_2)
    
    y_pred_test_3 = exported_pipeline.predict(X_test_3_sel)
    rmse_3 = np.sqrt(mean_squared_error(y_test_3, y_pred_test_3))
    r2_3 = r2_score(y_test_3, y_pred_test_3)
    pearson_corr_3, _ = pearsonr(y_test_3.values.flatten(), y_pred_test_3.flatten())
    print('score for test 3 - RMSE = ', rmse_3, ' - R2 = ', r2_3, ' - Pearson = ', pearson_corr_3)
    
    #Run 10-fold CV with results params
    exported_pipeline = tpot.fitted_pipeline_
    set_param_recursive(exported_pipeline.steps, 'random_state', 42)

    kfold = KFold(n_splits=10, shuffle=True, random_state=42)

    rmse_scores = []
    r2_scores = []
    pearson_corrs = []
    
    for train_index, test_index in kfold.split(X_train_sel[feat_list.values.flatten()]):
        X_train_fold, X_test_fold = X_train_sel[feat_list.values.flatten()].loc[train_index], X_train_sel[feat_list.values.flatten()].loc[test_index]
        y_train_fold, y_test_fold = y_train.loc[train_index], y_train.loc[test_index]
        
        # Train the model on the current fold
        exported_pipeline.fit(X_train_fold, y_train_fold)
        y_pred_fold = exported_pipeline.predict(X_test_fold)
        
        # Calculate RMSE, R2, and Pearson correlation
        rmse = np.sqrt(mean_squared_error(y_test_fold, y_pred_fold))
        r2 = r2_score(y_test_fold, y_pred_fold)
        pearson_corr, _ = pearsonr(y_test_fold.values.flatten(), y_pred_fold.flatten())
        
        # Store the scores
        rmse_scores = np.append(rmse_scores, rmse)
        r2_scores = np.append(r2_scores, r2)
        pearson_corrs = np.append(pearson_corrs, pearson_corr)


    #Have some code to print out all results as a doc file.
    results.append({
            "Method": "TPOT",
            "Feat set": feat_name,
            "Model params": exported_pipeline,
            "CV Mean RMSE (10-fold)": round(np.mean(rmse_scores), 3), 
            "CV Std RMSE (10-fold)": round(np.std(rmse_scores), 3),
            "CV Mean R2 (10-fold)": round(np.mean(r2_scores), 3),
            "CV Std R2 (10-fold)": round(np.std(r2_scores), 3),
            "CV Mean Pearson (10-fold)": round(np.mean(pearson_corrs), 3),
            "CV Std Pearson (10-fold)": round(np.std(pearson_corrs), 3),
            "Test 1 RMSE": round(rmse_1, 3),
            "Test 1 R2"  : round(r2_1, 3),
            "Test 1 Pearson" : round(pearson_corr_1, 3),
            "Test 2 RMSE": round(rmse_2, 3),
            "Test 2 R2"  : round(r2_2, 3),
            "Test 2 Pearson" : round(pearson_corr_2, 3),
            "Test 3 RMSE": round(rmse_3, 3),
            "Test 3 R2"  : round(r2_3, 3),
            "Test 3 Pearson" : round(pearson_corr_3, 3)
            })
    results_df = pd.DataFrame(results)
    
    results_df_tpot_save_path = result_path + 'TPOT_ddmut_results.csv'
    results_df.to_csv(results_df_tpot_save_path)

running - feat_name:  all_feat


Version 0.12.1 of tpot is outdated. Version 0.12.2 was released Friday February 23, 2024.


Optimization Progress:   0%|                                                                                                    | 0/100 [00:00<?, ?pipeline/s]

/home/korawich/anaconda3/envs/atml_TPOT/lib/python3.8/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
Traceback (most recent call last):
  File "_ctypes/callbacks.c", line 237, in 'calling callback function'
  File "/home/korawich/anaconda3/envs/atml_TPOT/lib/python3.8/site-packages/xgboost/core.py", line 594, in _next_wrapper
    def _next_wrapper(self, this: None) -> int:  # pylint: disable=unused-argument
stopit.utils.TimeoutException
/home/korawich/anaconda3/envs/atml_TP

                                                                                                                                                              
414.02 minutes have elapsed. TPOT will close down.                                                                                                            
TPOT closed during evaluation in one generation.
                                                                                                                                                              
                                                                                                                                                              
TPOT closed prematurely. Will use the current best pipeline.
                                                                                                                                                              
Best pipeline: LassoLarsCV(input_matrix, normalize=True)
score for test 1 - RMSE =  1.278214095

...

                                                                                                                                                              
Generation 15 - Current best internal CV score: -3.649989138479215
                                                                                                                                                              
Generation 16 - Current best internal CV score: -3.649989138479215
                                                                                                                                                              
Generation 17 - Current best internal CV score: -3.649989138479215
                                                                                                                                                              
Generation 18 - Current best internal CV score: -3.649989138479215
Optimization Progress: 100%|████████████████████████████████████████████████████████████████████

/home/korawich/anaconda3/envs/atml_TPOT/lib/python3.8/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/home/korawich/anaconda3/envs/atml_TPOT/lib/python3.8/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2

                                                                                                                                                              
Generation 19 - Current best internal CV score: -3.3707869806177584
Optimization Progress: 2101pipeline [3:56:50,  7.89s/pipeline]                                                                                                

/home/korawich/anaconda3/envs/atml_TPOT/lib/python3.8/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/home/korawich/anaconda3/envs/atml_TPOT/lib/python3.8/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2

                                                              
Generation 20 - Current best internal CV score: -3.3707869806177584
                                                              
241.25 minutes have elapsed. TPOT will close down.            
TPOT closed during evaluation in one generation.
                                                              
                                                              
TPOT closed prematurely. Will use the current best pipeline.
                                                              
Best pipeline: LinearSVR(Nystroem(input_matrix, gamma=1.0, kernel=additive_chi2, n_components=1), C=20.0, dual=True, epsilon=0.1, loss=squared_epsilon_insensitive, tol=0.001)
score for test 1 - RMSE =  1.4365070821583066  - R2 =  -0.037733490364811884  - Pearson =  nan
score for test 2 - RMSE =  1.927065861942934  - R2 =  -0.02062188934835607  - Pearson =  nan
score for test 3 - RMSE =  2.436391069647543  - R2 =  -0.012802281211706346  - Pe

In [19]:
results_df

,Method,Feat set,Model params,CV Mean RMSE (10-fold),CV Std RMSE (10-fold),CV Mean R2 (10-fold),CV Std R2 (10-fold),CV Mean Pearson (10-fold),CV Std Pearson (10-fold),Test 1 RMSE,Test 1 R2,Test 1 Pearson,Test 2 RMSE,Test 2 R2,Test 2 Pearson,Test 3 RMSE,Test 3 R2,Test 3 Pearson
0,TPOT,all_feat,(LassoLarsCV(normalize=True)),1.513,0.051,0.378,0.021,0.620,0.019,1.278,0.178,0.478,1.650,0.252,0.502,2.402,0.016,0.253
1,TPOT,seq_feat,(StackingEstimator(estimator=ExtraTreesRegress...,1.654,0.042,0.256,0.021,0.522,0.028,1.238,0.229,0.479,1.659,0.243,0.515,2.409,0.010,0.171
2,TPOT,str_feat,(FeatureUnion(transformer_list=[('functiontran...,1.269,0.044,0.562,0.024,0.753,0.017,1.181,0.298,0.594,1.566,0.326,0.571,2.251,0.135,0.392
3,TPOT,gbs_feat,"(Nystroem(gamma=1.0, kernel='additive_chi2', n...",2.284,0.411,-0.479,0.639,NaN,NaN,1.437,-0.038,NaN,1.927,-0.021,NaN,2.436,-0.013,NaN
